In [15]:
import lmdb
import numpy as np
import cv2
from caffe.proto.caffe_pb2 import Datum
from tqdm import tqdm
import os

In [16]:
env = lmdb.open('./mnist_train_lmdb/', map_size=80000*28*28*5)

In [17]:
img_dir = '../../buff_benchmark/red_number_new/'

In [18]:
with env.begin() as txn:
    print txn.stat()

{'branch_pages': 68L, 'leaf_pages': 15014L, 'overflow_pages': 2048L, 'psize': 4096L, 'depth': 3L, 'entries': 62048L}


In [19]:
with env.begin(write=True) as txn:
    i = 60000
    for img_name in tqdm(os.listdir(img_dir)):
        if not img_name.endswith('.jpg'):
            continue
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        datum = Datum()
        datum.data = img.tobytes()
        datum.width = 28
        datum.height = 28
        datum.channels = 1
        datum.label = int(img_name.split('_')[1].split('.')[0])
        txn.put('{:08}'.format(i), datum.SerializeToString())
        i += 1

100%|██████████| 2178/2178 [00:00<00:00, 11409.84it/s]


In [21]:
with env.begin() as txn:
    print txn.stat()

{'branch_pages': 71L, 'leaf_pages': 15436L, 'overflow_pages': 0L, 'psize': 4096L, 'depth': 3L, 'entries': 62048L}
